In [5]:
from collections import defaultdict
from razdel import tokenize
import json

In [2]:
# загружаем данные датасета nerus 
with open('nerus_lenta.conllu') as myfile:
    data = myfile.readlines()

In [6]:
# составим словарь предложение: (NER, тип NER)
sents = defaultdict(list)
temp = []
curr_sent = ''
for el in data:
    if '# text = ' in el:
        if temp:
            sents[curr_sent].append(temp)
            temp = []
        curr_sent = el[9:]
    splits = el.split('\t')
    for ell in splits:
        if ell == 'Tag=I-ORG\n' or  ell == 'Tag=I-PER\n' or  ell == 'Tag=I-LOC\n':
            temp.append((splits[1],splits[-1]))
        if ell == 'Tag=B-ORG\n' or  ell == 'Tag=B-PER\n' or  ell == 'Tag=B-LOC\n':
            if not temp:
                temp.append((splits[1],splits[-1]))
            else:
                sents[curr_sent].append(temp)
                temp = []
                temp.append((splits[1],splits[-1]))

In [7]:
# приведём словарь sents в вид:
# {‘sentence’: [лист токенов предложения], 
#     ‘ner’: [{
#          ‘index’: [индекс именованной сущности], 
#          ‘type’: наименование именованной сущности}]}

list_for_json = []
for k, v in sents.items():
    temp = {}
    temp['sentence'] = list(map(lambda x: x.text, tokenize(k))) 
    if len(temp['sentence']) > 100:
        continue
    temp_ner = []
    for ner in v:
        d = {}
        ixs = []
        type_ = ''
        for NER, typ in ner:
            ixs.append(temp['sentence'].index(NER))
            type_ =  typ[6:9]
        d['index'] = ixs
        d['type'] = type_
        temp_ner.append(d)
    temp['ner'] = temp_ner
    list_for_json.append(temp)

In [8]:
# сохраним данные в json файл
with open('parsed_data.json', 'w', encoding='utf-8') as fp:
    json.dump(list_for_json, fp, ensure_ascii=False)